In [1]:
import sys
sys.path.append("..")
import numpy as np 
import torch
from relie.utils.so3_tools import so3_hat, so3_vee, so3_exp, so3_log
from relie.utils.se3_tools import se3_hat, se3_vee, se3_exp, se3_log, se3_inv

In [2]:
def det_jac_so3(x):
    theta = x.norm(2,-1)
    return 2*(1 - torch.cos(theta))/(theta**2)

def so3_inv(el):
    return el.transpose(-2,-1)
    

In [3]:
def compute_approx_jacobian(points, vee, exp, log, inv, eps = 0.01):
    points = points.unsqueeze(-2)
    dim = points.shape[-1]
    basis = torch.eye(dim, dtype=torch.float64).unsqueeze(0)
    group_delta = exp(basis*eps + points)
    points_inv = inv(exp(points))
    normal_coord = vee(log(points_inv@group_delta))
    estimated_det_jac = np.linalg.det(normal_coord.numpy())/((eps)**dim)
    return torch.tensor(estimated_det_jac)

### SO(3)

In [4]:
def approximate_so3_jacobian(points, eps = 0.01):
    return compute_approx_jacobian(points, so3_vee, so3_exp, so3_log, so3_inv, eps)

In [5]:
dim = 3
eps = 0.005

center = torch.tensor(np.random.normal(0,1,(10,dim)),dtype = torch.float64)
estimated_det_jac = approximate_so3_jacobian(center, eps=eps)
#print(estimated_det_jac - det_jac_so3(center))

### SE(3)

In [6]:
def approximate_se3_jacobian(points, eps = 0.01):
    return compute_approx_jacobian(points, se3_vee, se3_exp, se3_log, se3_inv, eps)

In [7]:
dim = 6
eps = 0.1

center = torch.tensor(np.random.normal(0,1,(10,dim)),dtype = torch.float64)
estimated_det_jac = approximate_se3_jacobian(center, eps=eps)

tensor([[[0.0921],
         [0.0994],
         [0.0925],
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.0993],
         [0.0995],
         [0.0989],
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.0980],
         [0.0993],
         [0.0981],
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.0982],
         [0.0994],
         [0.0986],
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.0895],
         [0.0867],
         [0.0957],
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.0866],
         [0.0947],
         [0.0818],
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.0947],
         [0.0971],
         [0.0935],
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.0966],
         [0.0970],
         [0.0993],
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.0979],
         [0.0986],
         [0.0971],
         [0.000

/home/luca/anaconda2/envs/ds/lib/python3.5/site-packages/numpy/linalg/linalg.py:2010: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [8]:
theta = torch.tensor(np.random.normal(0,0.1,(10,dim)),dtype = torch.float64)
((1 - theta*torch.sin(theta))/(2-2*torch.cos(theta)))/theta**2

tensor([[6.2358e+03, 1.6871e+04, 3.2478e+07, 2.2302e+03, 1.7353e+03, 4.3283e+03],
        [1.9454e+06, 7.0163e+03, 5.8126e+06, 5.8743e+05, 9.3346e+03, 1.3267e+03],
        [8.0370e+03, 8.2796e+05, 1.1232e+04, 3.5107e+03, 1.7640e+04, 2.0147e+03],
        [1.1372e+03, 6.6834e+04, 7.0387e+05, 4.4974e+08, 1.7241e+06, 2.4425e+02],
        [1.9968e+03, 7.2730e+03, 1.6255e+05, 3.8968e+07, 1.4362e+03, 2.6949e+05],
        [9.4137e+03, 2.1755e+04, 1.0536e+04, 1.3331e+04, 3.5022e+04, 1.2055e+05],
        [2.1299e+03, 1.0771e+05, 8.5527e+02, 1.2495e+03, 1.2627e+07, 2.9380e+05],
        [9.1207e+02, 1.3703e+07, 1.4626e+09, 8.2159e+02, 1.9024e+03, 1.8001e+10],
        [1.4945e+04, 5.7412e+03, 4.0167e+03, 4.7756e+05, 4.0886e+04, 4.8041e+04],
        [1.5230e+04, 2.0554e+03, 3.0993e+05, 4.1709e+05, 9.0155e+04, 6.4451e+02]],
       dtype=torch.float64)

In [ ]:
x = so3_log(torch.eye(3, dtype = torch.float64).unsqueeze(0)) 
x

In [ ]:
mask_nan = x != x

In [ ]:
x[mask_nan] = 0

In [ ]:
x